# Load dependencies

In [1]:
#load dependencies
#load in dependencies to read data
using CSV, Plots, StatsPlots, DataFrames, Dates
using DifferentialEquations,LaTeXStrings,LinearAlgebra,Pkg
#show versions
Pkg.status()

    Status `C:\Users\Steve\.julia\environments\v1.3\Project.toml`
  [336ed68f] CSV v0.5.26
  [a93c6f00] DataFrames v0.20.2
  [aae7a2af] DiffEqFlux v1.3.2
  [0c46a032] DifferentialEquations v6.10.1
  [587475ba] Flux v0.10.1
  [7073ff75] IJulia v1.21.1
  [c601a237] Interact v0.10.3
  [b964fa9f] LaTeXStrings v1.1.0
  [47be7bcc] ORCA v0.3.1
  [429524aa] Optim v0.20.1
  [f0f68f2c] PlotlyJS v0.13.1
  [91a5bcdd] Plots v0.28.4
  [d330b81b] PyPlot v2.8.2
  [6f49c342] RCall v0.13.4
  [90137ffa] StaticArrays v0.12.1
  [f3b207a7] StatsPlots v0.14.2
  [fd094767] Suppressor v0.1.1
  [30578b45] URIParser v0.4.0
  [0ae4a718] VegaDatasets v2.0.0
  [112f6efa] VegaLite v2.0.1
  [0f1e0344] WebIO v0.8.13
  [fdbf4ff8] XLSX v0.6.0


# Spatial Epidemic Model for GA for COVID-19

Here, we extend the model to look at infection spread through a spatial metapopulation model.
To do this, we consider that individuals spend time in the local community, but also may commute - for work, errands, social activities or other things. Following this we split the population $N_{tot}$ into two compartments $N_{local}$ and $N_{commute}$. $N_{local}$ is the population who live in a particular county, whilst $N_{commute} = N_{local} - N_{leave} + N_{arrive}$. Here $N_{leave}$ are those who leave the county, whilst $N_{arrive}$ are those who travel into the county. We do not explicitly account for those arriving or leaving in the local ODE's.

We assume that those in the county see the local population for a proportion of time $w_{local}$ and the commuter population for a proportion of time $w_{commmute}$, where we assume $w_{local} + w_{commute} = 1$.


The system of nonlinear differential equations governing this patch and age-structured epidemiological for patch $i$ and age $a$ are as follows:

\begin{align}
\frac{dS(i,a)}{dt} & =  -\overset{new\ cases\ from\ symptomatic\ contact}{\overbrace{\beta_S S(i,a) \left( w_{local}\frac{ I_{S,local}(i)}{N_{local}(i)} + w_{commute}\frac{ I_{S,commute}(i)}{N_{commute}(i)} \right) }}  - \overset{new\ cases\ from\ asymptomatic\ contact}{\overbrace{\beta_A S(i,a) \left( w_{local}\frac{ I_{A,local}(i)}{N_{local}(i)} + w_{commute}\frac{ I_{A,commute}(i)}{N_{commute}(i)} \right) }}  \\ \\
\frac{dE(i,a)}{dt} & =  \overset{new\ cases\ from\ symptomatic\ contact}{\overbrace{\beta_S S(i,a) \left( w_{local}\frac{ I_{S,local}(i)}{N_{local}(i)} + w_{commute}\frac{ I_{S,commute}(i)}{N_{commute}(i)} \right) }}  + \overset{new\ cases\ from\ asymptomatic\ contact}{\overbrace{\beta_A S(i,a) \left( w_{local}\frac{ I_{A,local}(i)}{N_{local}(i))} + w_{commute}\frac{ I_{A,commute}(i)}{N_{commute}(i)} \right) }}  - \overset{onset\ of\ infectiousness}{\overbrace{\gamma_E E(i,a)}} \\
\frac{dI_A(i,a)}{dt} & =  \overset{asymptomatic\ onset}{\overbrace{p\gamma_E E(i,a)}} - \overset{recovery}{\overbrace{\gamma_A I_A(i,a)}} \\
\frac{dI_S(i,a)}{dt} & =  \overset{symptomatic\ onset}{\overbrace{(1-p)\gamma_E E(i,a)}} - \overset{transfer\ from\ I_S}{\overbrace{\gamma_S I_S(i,a)}} \\
\frac{dI_{h,sub}(i,a)}{dt} & =  \overset{subcritical\ cases}{\overbrace{h(a)(1-\xi(a)) \gamma_S I_S(i,a)}} - \overset{transfer\ from\ I_{h,sub}}{\overbrace{\gamma_h I_{h,sub}(i,a)}} \\
\frac{dI_{h,crit}(i,a)}{dt} & =  \overset{critical\ (ICU)\ cases}{\overbrace{h(a)\xi(a) \gamma_S I_S(i,a)}} - \overset{transfer\ from\ I_{h,crit}}{\overbrace{\gamma_h I_{h,crit}(i,a)}} \\
\frac{dR(a)}{dt} & =  \overset{recovery\ from\ I_A}{\overbrace{\gamma_A I_A(i,a)}} - \overset{recovery\ from\ I_S}{\overbrace{(1-h(a))\gamma_S I_S(i,a)}} - \overset{recovery\ from\ I_{h,sub}}{\overbrace{\gamma_h I_{h,sub}(i,a)}} - \overset{recovery\ from\ I_{h,crit}}{\overbrace{(1-\mu)\gamma_h I_{h,crit}(i,a)}} \\
\frac{dD(i,a)}{dt} & = \overset{mortality}{\overbrace{\mu \gamma_h I_{h,crit}(i,a)}}
\end{align}

where $I_{S,local}$ and $I_{S,commute}$ are the total number (across all age classes) of sympomatic infectious individuals; $I_{A,local}$ and $I_{S,commute}$ are the total number of asymptomatic infectious individuals; and $N_{local}$ and $N_{commute}$ are the total number of alive individuals during local and commute times.

# Read in functions called in the scripts

In [2]:
#ODE model code for spatial model
#NOTE: this is no longer called as it assumes the transport between counties is age-independent(keeping as archive)
function SpatialAgeStructuredEpidemiologicalModel!(du,u,par,t)
    SDI,β_a,β_s,p,γ_e,γ_a,γ_s,γ_h,h,ξ,μ,S_ind,E_ind,
    Ia_ind,Is_ind,Ihsub_ind,Ihcrit_ind,R_ind,D_ind,
    NumPatches,NumAgeClasses,patchstates,patchstatesALIVE,
    patchstatesMOVEABLE,w,transport,ISMOVEFACTOR,w1,w2 = par
    
    #NumPatches is the number of spatial "nodes"
    #w is a matrix of size NxN determining flows between nodes.
    #transport is a matrix to store movements vectors (don't want to recreate this object throughout this function)
    #patchstates - total number of states in a patch
    #patchstatesALIVE - number of non-dead states
    #patchstatesMOVEABLE - number of states that are allowed to move between patches
    #S_ind,E_ind,Ia_ind,Is_ind,Ihsub_ind,Ihcrit_ind,R_ind,D_ind - indexes of the states for the first patch i.e. number 1:80

    
    #have matrix w, representing movement of commuters i.e. X1 stay in county, X2 go to county 2, X3 go to county3
    #total travel is fraction travelled applied to people in moving states
    #some amounts of these will be IA and IS - keep track!
    #want to know N,Is,IA leaving each county; and arriving at each county
    #w11 S1 + w12 S2  #net change for each state so long as diagonal is -ve.
        
    #Stage 1:
    #calculate transport between patches
    
    #transport = transport .* 0.0;
    #want this to be age structured
    for ii = 1:patchstatesMOVEABLE #for each age class of alive states that can move.
        INDS = ii:patchstates:(ii+(NumPatches-1)*patchstates ) #gives 1xpatch indices for transport multiplication for each age class and each state that can move
        transport[:,ii] =  w*u[INDS]  #add the 1*patch vector of movement fluxes to transport
        #mul!(transport[:,ii],w,u[INDS])
    end
    
    #modify tranport matrix to account for Is people deciding to move or not.
    transport[:,Is_ind] = ISMOVEFACTOR .* transport[:,Is_ind]

    #calculate net change for each patch for N,Ia and Is
    C_TOT_NETCHANGE = sum(transport,dims=2)
    C_I_A_NETCHANGE = sum(transport[:,Ia_ind],dims=2)
    C_I_S_NETCHANGE = sum(transport[:,Is_ind],dims=2)
    
    #Stage 2:
    #Go through the ODE system node by node. As have found fluxes between nodes already, this will be a synchronous update of local and spatial dynamics.

    for aa=1:(Int64(NumPatches)) #for each patch node
        
        #Local patch dynamics depend on patch quantitites
        NTOT_Loc = sum(u[((aa-1)*patchstates+1):((aa-1)*patchstates+patchstatesALIVE)]) #total alive
        TOTI_a_Loc = sum(u[(aa-1)*patchstates .+ Ia_ind]) #total asymptomatic infected
        TOTI_s_Loc = sum(u[(aa-1)*patchstates .+ Is_ind]) #total symptomatic infected
        
        NTOT_Com = NTOT_Loc + C_TOT_NETCHANGE[aa]#total alive
        TOTI_a_Com = TOTI_a_Loc + C_I_A_NETCHANGE[aa] #total asymptomatic infected
        TOTI_s_Com = TOTI_s_Loc + C_I_S_NETCHANGE[aa] #total symptomatic infected
        
        LOCAL = w1/(NTOT_Loc)
        COMMUTE = w2/(NTOT_Com)
        propI_A = ( TOTI_a_Loc*LOCAL + TOTI_a_Com*COMMUTE )
        propI_S = ( TOTI_s_Loc*LOCAL + TOTI_s_Com*COMMUTE )
        
        
        for bb = 1:(Int64(NumAgeClasses))  #for each age class in this patch
        INDS = (aa-1)*patchstates .+ [S_ind[bb],E_ind[bb],Ia_ind[bb],Is_ind[bb],R_ind[bb],Ihsub_ind[bb],Ihcrit_ind[bb],D_ind[bb]]   
            
            
            S,E,Ia,Is,R,Ihsub,Ihcrit,D = u[INDS]

            du[INDS[1]] =  -β_a*SDI*S*propI_A - β_s*SDI*S*propI_S   #dS/dt
            du[INDS[2]] =  +β_a*SDI*S*propI_A + β_s*SDI*S*propI_S - γ_e*E  #dE/dt
            du[INDS[3]] = p[bb]*γ_e*E - γ_a*Ia   #dI_a /dt
            du[INDS[4]] = (1-p[bb])*γ_e*E - γ_s*Is  #dI_s/dt
            du[INDS[5]] = γ_a*Ia + (1-h[bb])*γ_s*Is + γ_h*Ihsub + (1-μ[bb])*γ_h*Ihcrit #dR/dt
            du[INDS[6]] = h[bb]*(1-ξ[bb])*γ_s*Is - γ_h*Ihsub #dIhsub/dt
            du[INDS[7]] = h[bb]*ξ[bb]*γ_s*Is -  γ_h*Ihcrit #dIhcrit/dt
            du[INDS[8]] = μ[bb]*γ_h*Ihcrit #dD/dt
        end
    end
    
end


SpatialAgeStructuredEpidemiologicalModel! (generic function with 1 method)

In [3]:
#ODE model code
#assumes age-dependent journeys between counties
function SpatialAgeStructuredEpidemiologicalModelMixAge!(du,u,par,t)
    SDI,β_a,β_s,p,γ_e,γ_a,γ_s,γ_h,h,ξ,μ,S_ind,E_ind,
    Ia_ind,Is_ind,Ihsub_ind,Ihcrit_ind,R_ind,D_ind,
    NumPatches,NumAgeClasses,patchstates,patchstatesALIVE,
    patchstatesMOVEABLE,w,wy,wo,transport,ISMOVEFACTOR,w1,w2 = par
    
    #NumPatches is the number of spatial "nodes"
    #w is a matrix of size NxN determining flows between nodes.
    #transport is a matrix to store movements vectors (don't want to recreate this object throughout this function)
    #patchstates - total number of states in a patch
    #patchstatesALIVE - number of non-dead states
    #patchstatesMOVEABLE - number of states that are allowed to move between patches
    #S_ind,E_ind,Ia_ind,Is_ind,Ihsub_ind,Ihcrit_ind,R_ind,D_ind - indexes of the states for the first patch i.e. number 1:80

    
    #have matrix w, representing movement of commuters i.e. X1 stay in county, X2 go to county 2, X3 go to county3
    #total travel is fraction travelled applied to people in moving states
    #some amounts of these will be IA and IS - keep track!
    #want to know N,Is,IA leaving each county; and arriving at each county
    #w11 S1 + w12 S2  #net change for each state so long as diagonal is -ve.

    #w is those aged 30-59
    #wy is those 0-29
    #wo is those 60+
    
    #Stage 1:
    #calculate transport between patches
    
    #transport = transport .* 0.0;
    #want this to be age structured
    for ii = 1:patchstatesMOVEABLE #for each age class of alive states that can move.
        INDS = ii:patchstates:(ii+(NumPatches-1)*patchstates ) #gives 1xpatch indices for transport multiplication for each age class and each state that can move
        MOD = mod(ii,NumAgeClasses-1)
        if MOD>0 & MOD<4
        transport[:,ii] = wy*u[INDS] 
        elseif MOD==0  || MOD >=7
        transport[:,ii] = wo*u[INDS]
        else
        transport[:,ii] =  w*u[INDS]  #add the 1*patch vector of movement fluxes to transport
        #mul!(transport[:,ii],w,u[INDS])
        end
    end
    
    #modify tranport matrix to account for Is people deciding to move or not.
    transport[:,Is_ind] = ISMOVEFACTOR .* transport[:,Is_ind]

    #calculate net change for each patch for N,Ia and Is
    C_TOT_NETCHANGE = sum(transport,dims=2)
    C_I_A_NETCHANGE = sum(transport[:,Ia_ind],dims=2)
    C_I_S_NETCHANGE = sum(transport[:,Is_ind],dims=2)
    
    #Stage 2:
    #Go through the ODE system node by node. As have found fluxes between nodes already, this will be a synchronous update of local and spatial dynamics.

    for aa=1:(Int64(NumPatches)) #for each patch node
        
        #Local patch dynamics depend on patch quantitites
        NTOT_Loc = sum(u[((aa-1)*patchstates+1):((aa-1)*patchstates+patchstatesALIVE)]) #total alive
        TOTI_a_Loc = sum(u[(aa-1)*patchstates .+ Ia_ind]) #total asymptomatic infected
        TOTI_s_Loc = sum(u[(aa-1)*patchstates .+ Is_ind]) #total symptomatic infected
        
        NTOT_Com = NTOT_Loc + C_TOT_NETCHANGE[aa]#total alive
        TOTI_a_Com = TOTI_a_Loc + C_I_A_NETCHANGE[aa] #total asymptomatic infected
        TOTI_s_Com = TOTI_s_Loc + C_I_S_NETCHANGE[aa] #total symptomatic infected
        
        LOCAL = w1/(NTOT_Loc)
        COMMUTE = w2/(NTOT_Com)
        propI_A = ( TOTI_a_Loc*LOCAL + TOTI_a_Com*COMMUTE )
        propI_S = ( TOTI_s_Loc*LOCAL + TOTI_s_Com*COMMUTE )
        
        
        for bb = 1:(Int64(NumAgeClasses))  #for each age class in this patch
        INDS = (aa-1)*patchstates .+ [S_ind[bb],E_ind[bb],Ia_ind[bb],Is_ind[bb],R_ind[bb],Ihsub_ind[bb],Ihcrit_ind[bb],D_ind[bb]]   
            
            
            S,E,Ia,Is,R,Ihsub,Ihcrit,D = u[INDS]

            du[INDS[1]] =  -β_a*SDI*S*propI_A - β_s*SDI*S*propI_S   #dS/dt
            du[INDS[2]] =  +β_a*SDI*S*propI_A + β_s*SDI*S*propI_S - γ_e*E  #dE/dt
            du[INDS[3]] = p[bb]*γ_e*E - γ_a*Ia   #dI_a /dt
            du[INDS[4]] = (1-p[bb])*γ_e*E - γ_s*Is  #dI_s/dt
            du[INDS[5]] = γ_a*Ia + (1-h[bb])*γ_s*Is + γ_h*Ihsub + (1-μ[bb])*γ_h*Ihcrit #dR/dt
            du[INDS[6]] = h[bb]*(1-ξ[bb])*γ_s*Is - γ_h*Ihsub #dIhsub/dt
            du[INDS[7]] = h[bb]*ξ[bb]*γ_s*Is -  γ_h*Ihcrit #dIhcrit/dt
            du[INDS[8]] = μ[bb]*γ_h*Ihcrit #dD/dt
        end
    end
    
end


SpatialAgeStructuredEpidemiologicalModelMixAge! (generic function with 1 method)

In [4]:
#ODE model code - this is used to model at the level of the whole of Georgia! (helpful in setting initial conditions)
function SingleNode!(du,u,par,t)
    SDI,β_a,β_s,p,γ_e,γ_a,γ_s,γ_h,h,ξ,μ,S_ind,E_ind,Ia_ind,Is_ind,R_ind,Ihsub_ind,Ihcrit_ind,D_ind = par
    
    TOTI_a = sum(u[Ia_ind]) #Total asymptomatic infectious individuals
    TOTI_s = sum(u[Is_ind]) #Total symptomatic infectious individuals

    
    NTOT = sum(u[1:63]) #All individuals not in the dead (D) state

    
    for aa = 1:length(S_ind)#for each age class
        INDS = [S_ind[aa],E_ind[aa],Ia_ind[aa],Is_ind[aa],R_ind[aa],Ihsub_ind[aa],Ihcrit_ind[aa],D_ind[aa]]
        S,E,Ia,Is,R,Ihsub,Ihcrit,D = u[INDS]
        du[INDS[1]] = -β_a*SDI*S*TOTI_a/(NTOT) - β_s*SDI*S*TOTI_s/(NTOT) #dS/dt
        du[INDS[2]] = +β_a*SDI*S*TOTI_a/(NTOT) + β_s*SDI*S*TOTI_s/(NTOT) -  γ_e*E   #dE/dt
        du[INDS[3]] = p[aa]*γ_e*E - γ_a*Ia  #dI_a /dt
        du[INDS[4]] = (1-p[aa])*γ_e*E - γ_s*Is  #dI_s/dt
        du[INDS[5]] = γ_a*Ia + (1-h[aa])*γ_s*Is + γ_h*Ihsub + (1-μ[aa])*γ_h*Ihcrit  #dR/dt
        du[INDS[6]] = h[aa]*(1-ξ[aa])*γ_s*Is - γ_h*Ihsub #dIhsub/dt
        du[INDS[7]] = h[aa]*ξ[aa]*γ_s*Is -  γ_h*Ihcrit #dIhcrit/dt
        du[INDS[8]] = μ[aa]*γ_h*Ihcrit #dD/dt
    end
 
end

SingleNode! (generic function with 1 method)

In [5]:
#reads in ODE solution; and saves output summary figures and CSV files
function WriteOutput!(PatchFIPS,PatchNames,GAPOP,TODAY,TIMERESOLTION,T_END,sol,pars,SaveName)
    SDI,β_a,β_s,p,γ_e,γ_a,γ_s,γ_h,h,ξ,μ,S_ind,E_ind,
        NumPatches,NumAgeClasses,patchstates,patchstatesALIVE,
        patchstatesMOVEABLE,w,wy,wo,transport,ISMOVEFACTOR,w1,w2 = pars 
    
    
    STATES = GAPOP * gatherStates!(sol,pars) #organise the solution
    #patches  x EpiStates x AgeClasses x Time
    SIZES = size(STATES) 
    
    #Make summary tables for each epi state, and total hospitalised
    #S,E,Ia,Is,R,Ihsub,Ihcrit,D, Ihsub+Ihcrit
    S = zeros(SIZES[1],SIZES[4]) # pacthes x time
    E = zeros(SIZES[1],SIZES[4]) # pacthes x time
    Ia = zeros(SIZES[1],SIZES[4]) # pacthes x time
    Is = zeros(SIZES[1],SIZES[4]) # pacthes x time
    R = zeros(SIZES[1],SIZES[4]) # pacthes x time
    Ihsub = zeros(SIZES[1],SIZES[4]) # pacthes x time
    Ihcrit = zeros(SIZES[1],SIZES[4]) # pacthes x time
    D = zeros(SIZES[1],SIZES[4]) # pacthes x time
    Htot = zeros(SIZES[1],SIZES[4]) # pacthes x time
    Dperday = zeros(SIZES[1],SIZES[4]) # pacthes x time
    NewCperday = zeros(SIZES[1],SIZES[4]) # pacthes x time
    
    for aa = 1:SIZES[1] #for each patch
        S[aa,:] = dropdims(sum(STATES[aa,1,1:9,:],dims=1),dims=1)
        E[aa,:] = dropdims(sum(STATES[aa,2,1:9,:],dims=1),dims=1)
        Ia[aa,:] = dropdims(sum(STATES[aa,3,1:9,:],dims=1),dims=1)
        Is[aa,:] = dropdims(sum(STATES[aa,4,1:9,:],dims=1),dims=1)
        R[aa,:] = dropdims(sum(STATES[aa,5,1:9,:],dims=1),dims=1)
        Ihsub[aa,:] = dropdims(sum(STATES[aa,6,1:9,:],dims=1),dims=1)
        Ihcrit[aa,:] = dropdims(sum(STATES[aa,7,1:9,:],dims=1),dims=1)
        D[aa,:] = dropdims(sum(STATES[aa,8,1:9,:],dims=1),dims=1)
        Htot[aa,:] =  Ihsub[aa,:] .+ Ihcrit[aa,:]
        for bb = 1:SIZES[3] #for age classes
            #Dperday[aa,:] .+=  μ[bb].*γ_h .*STATES[aa,7,bb,:].*1.0;  #deaths per day
            Dperday[aa,:] .+= [( STATES[aa,8,bb,2:end] .- STATES[aa,8,bb,1:(end-1)]) ./ (sol.t[2:end] - sol.t[1:(end-1)]); 0]
            NewCperday[aa,:] .+= [( STATES[aa,1,bb,1:(end-1)] .- STATES[aa,1,bb,2:end]) ./ (sol.t[2:end] - sol.t[1:(end-1)]); 0]
        end
    end
    
    #build Time column names
    #colnames = Array{String,1}(undef,SIZES[4])
    #for aa = 1:SIZES[4]
    #    colnames[aa] = string("T",sol.t[aa])
    #end
    
    #build Time column names v2
    #colnames = TODAY:Dates.Day(TIMERESOLUTION):(TODAY + Dates.Day(T_END))
    colnames = Array{Date,1}(undef,1)
    colnames[1] = TODAY
    for aa = 2:(SIZES[4])#for each time
        push!(colnames,colnames[aa-1]+Dates.Day(convert(Int,sol.t[aa]-sol.t[aa-1])))
    end

    

    #build dataframes
    SSdata = DataFrame(S)
    rename!(SSdata , Symbol.(colnames))
    insertcols!(SSdata, 1, :FIP => PatchFIPS)
    insertcols!(SSdata, 2, :County => PatchNames)
    
    EEdata = DataFrame(E)
    rename!(EEdata , Symbol.(colnames))
    insertcols!(EEdata, 1, :FIP => PatchFIPS)
    insertcols!(EEdata, 2, :County => PatchNames)
    
    Iadata = DataFrame(Ia)
    rename!(Iadata , Symbol.(colnames))
    insertcols!(Iadata, 1, :FIP => PatchFIPS)
    insertcols!(Iadata, 2, :County => PatchNames)
    
    Isdata = DataFrame(Is)
    rename!(Isdata , Symbol.(colnames))
    insertcols!(Isdata, 1, :FIP => PatchFIPS)
    insertcols!(Isdata, 2, :County => PatchNames)
   
    RRdata = DataFrame(R)
    rename!(RRdata , Symbol.(colnames))
    insertcols!(RRdata, 1, :FIP => PatchFIPS)
    insertcols!(RRdata, 2, :County => PatchNames)
    
    Ihsubdata = DataFrame(Ihsub)
    rename!(Ihsubdata , Symbol.(colnames))
    insertcols!(Ihsubdata, 1, :FIP => PatchFIPS)
    insertcols!(Ihsubdata, 2, :County => PatchNames)
    
    Ihcritdata = DataFrame(Ihcrit)
    rename!(Ihcritdata , Symbol.(colnames))
    insertcols!(Ihcritdata, 1, :FIP => PatchFIPS)
    insertcols!(Ihcritdata, 2, :County => PatchNames)
    
    DDdata = DataFrame(D)
    rename!(DDdata , Symbol.(colnames))
    insertcols!(DDdata, 1, :FIP => PatchFIPS)
    insertcols!(DDdata, 2, :County => PatchNames)
    
    HHdata = DataFrame(Htot)
    rename!(HHdata , Symbol.(colnames))
    insertcols!(HHdata, 1, :FIP => PatchFIPS)
    insertcols!(HHdata, 2, :County => PatchNames)
    
    DpDdata = DataFrame(Dperday)
    rename!(DpDdata , Symbol.(colnames))
    insertcols!(DpDdata, 1, :FIP => PatchFIPS)
    insertcols!(DpDdata, 2, :County => PatchNames)
    
    CpDdata = DataFrame(NewCperday)
    rename!(CpDdata , Symbol.(colnames))
    insertcols!(CpDdata, 1, :FIP => PatchFIPS)
    insertcols!(CpDdata, 2, :County => PatchNames)
    
    #make a directory to store files
    mkdir(SaveName)
    
    #write data to file
    CSV.write(string(SaveName,"/GA_county_Susceptibles.csv"),SSdata)
    CSV.write(string(SaveName,"/GA_county_Exposed.csv"),EEdata)
    CSV.write(string(SaveName,"/GA_county_AsymptInfected.csv"),Iadata)
    CSV.write(string(SaveName,"/GA_county_SymptInfected.csv"),Isdata)
    CSV.write(string(SaveName,"/GA_county_Recovered.csv"),RRdata)
    CSV.write(string(SaveName,"/GA_county_subacuteHospitalised.csv"),Ihsubdata)
    CSV.write(string(SaveName,"/GA_county_criticalHospitalised.csv"),Ihcritdata)
    CSV.write(string(SaveName,"/GA_county_Killed.csv"),DDdata)
    CSV.write(string(SaveName,"/GA_county_totalHospitalised.csv"),HHdata)
    CSV.write(string(SaveName,"/GA_county_deathrateperday.csv"),DpDdata)
    CSV.write(string(SaveName,"/GA_county_newcaserateperday.csv"),CpDdata)
    
    #Save summary data at County Level
    #sum across counties
    GASummary = zeros(11,SIZES[4])
    GASummary[1,:] = sum(S,dims=1)
    GASummary[2,:] = sum(E,dims=1)
    GASummary[3,:] = sum(Ia,dims=1)
    GASummary[4,:] = sum(Is,dims=1)
    GASummary[5,:] = sum(R,dims=1)
    GASummary[6,:] = sum(Ihsub,dims=1)
    GASummary[7,:] = sum(Ihcrit,dims=1)
    GASummary[8,:] = sum(D,dims=1)
    GASummary[9,:] = sum(Htot,dims=1)
    GASummary[10,:] = sum(Dperday,dims=1)
    GASummary[11,:] = sum(NewCperday,dims=1)
    
    GASum = DataFrame(GASummary)
    rename!(GASum , Symbol.(colnames))
    insertcols!(GASum, 1,:EpiState => ["S","E","Ia","Is","R","Ihsub","Ihcrit","D","Htot","Dperday","NCperday"])
    
    CSV.write(string(SaveName,"/GA_Summary.csv"),GASum)
    
    
end

WriteOutput! (generic function with 1 method)

In [6]:
#Used to turn the solution variable 1x(159*9*8) into an array that can be more easily parsed
function gatherStates!(sol,par)
    SDI,β_a,β_s,p,γ_e,γ_a,γ_s,γ_h,h,ξ,μ,S_ind,E_ind,
    Ia_ind,Is_ind,Ihsub_ind,Ihcrit_ind,R_ind,D_ind,
    NumPatches,NumAgeClasses,patchstates,patchstatesALIVE,
    patchstatesMOVEABLE,w,wy,wo,transport,ISMOVEFACTOR,w1,w2 = par
    
    Sols =  hcat(sol.u...)
    
    ARRAYSTORE = zeros(NumPatches,8,NumAgeClasses,length(sol.t)) #array of patches x epi states x ages x time
    
    for aa = 1:NumPatches
        for bb = 1:NumAgeClasses
        INDS = (aa-1)*patchstates .+ [S_ind[bb],E_ind[bb],Ia_ind[bb],Is_ind[bb],R_ind[bb],Ihsub_ind[bb],Ihcrit_ind[bb],D_ind[bb]]   
            for cc=1:8
            ARRAYSTORE[aa,cc,bb,:] = Sols[INDS[cc],:]
            end
        end
    end
    return ARRAYSTORE
end       
        

gatherStates! (generic function with 1 method)

In [7]:
#summary plot - plots the summed epi. states at the Georgia level
function SummaryPlot!(JAM,T)
SZ = size(JAM)
#AA=dropdims(sum(JAM[aa,1,1:9,:],dims=1),dims=1)[end]
S = zeros(SZ[4])
E = zeros(SZ[4])
Ia = zeros(SZ[4])
Is = zeros(SZ[4])
R = zeros(SZ[4])
Ihs = zeros(SZ[4])
Ihc = zeros(SZ[4])
D = zeros(SZ[4])

for aa = 1:SZ[1]
    S .+= dropdims(sum(JAM[aa,1,1:9,:],dims=1),dims=1)
    E .+= dropdims(sum(JAM[aa,2,1:9,:],dims=1),dims=1)
    Ia .+= dropdims(sum(JAM[aa,3,1:9,:],dims=1),dims=1)
    Is .+= dropdims(sum(JAM[aa,4,1:9,:],dims=1),dims=1)
    R .+= dropdims(sum(JAM[aa,5,1:9,:],dims=1),dims=1)
    Ihs .+= dropdims(sum(JAM[aa,6,1:9,:],dims=1),dims=1)
    Ihc .+= dropdims(sum(JAM[aa,7,1:9,:],dims=1),dims=1)
    D .+= dropdims(sum(JAM[aa,8,1:9,:],dims=1),dims=1)
end

pl = plot(T,S,label="S",yguidefontsize=10,xlabel="Time (days)",ylabel="People",legend=:outertopright)
plot!(T,E,label="E")
plot!(T,Ia,label="Ia")
plot!(T,Is,label="Is")
plot!(T,R,label="R")
plot!(T,Ihs,label="Ihsub")
plot!(T,Ihc,label="Ihcrit")
plot!(T,D,label="D")

return pl
end

SummaryPlot! (generic function with 1 method)

# Set up the model scripts

In [8]:
#Model states
NumPatches = 159 ; #159 counties
NumAgeClasses = 9; # 0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80+
NumEpiStates = 8; # S,E,Ia,Is,R,Ihsub,Ihcrit,D

patchstates = NumAgeClasses*NumEpiStates
patchstatesMOVEABLE = (NumEpiStates-3)*NumAgeClasses  #D cannot move. Ihsub, Ihcrit cannot move.
#Say that I_s may be able to move - can modify this assumption later.
patchstatesALIVE = (NumEpiStates-1)*NumAgeClasses #all but D are alive
transport = zeros(NumPatches,patchstatesMOVEABLE);

#Epidemiological state indexing - just for within one patch.
S_ind = 1:NumAgeClasses
E_ind = (1*NumAgeClasses+1):2*NumAgeClasses
Ia_ind = (2*NumAgeClasses+1):3*NumAgeClasses
Is_ind = (3*NumAgeClasses+1):4*NumAgeClasses
R_ind = (4*NumAgeClasses+1):5*NumAgeClasses
Ihsub_ind = (5*NumAgeClasses+1):6*NumAgeClasses
Ihcrit_ind = (6*NumAgeClasses+1):7*NumAgeClasses
D_ind = (7*NumAgeClasses+1):8*NumAgeClasses

#Age class information
MinAge = [0,10,20,30,40,50,60,70,80]
MiddleAge = [5,15,25,35,45,55,65,75,90];

In [35]:
#######################################STUFFTO EDIT ###################### 
#Time stepping stuff
TIMERESOLUTION = 7 #MAKE THIS AN INTEGER IN DAYS!!!!!! REQUIRED FOR SAVING LATER ON
T_END = 42
tspanrun = (0.0,T_END*1.0) #run the simulations between 0 days and 400 days
TODAY = Date(2020,4,9) #Date format of time 0

#Load in Data used to set time 0
CasesbyCounty = CSV.read("../state_data/casedatabyFIP/filtered/12pm_9Apr2020.csv");
#CasesbyCounty = CSV.read("../state_data/casedatabyFIP/filtered/28March2020.csv");

#String denoting where to save output
SAVEFOLD = "SimulationOutput/from28Mar_sim"

#initial condition setting to use (5 - uses a simulation method county by county; 7 - uses the state level model to partition to counties)
#other choices are archives right now from previous attempts
CHOICES = 5
#######################################STUFFTO EDIT ###################### 


#read in the GA level data

ODL29 = CSV.read("../state_data/ga_countyOD_29_younger.csv") # transport matrix for 0-29
OD30_55 = CSV.read("../state_data/ga_countyOD_30_54.csv") #transport matrix for 30-55 (assume till 59)
OD55_up = CSV.read("../state_data/ga_countyOD_55_older.csv") #transport matrix for 55+ (assume for 60+)
GA_county_size = CSV.read("../state_data/julia_read_data/GA_county_decade_age.csv") #age distributions by county
#GA_pop_ar =  CSV.read("../state_data/ga_county_population_area.csv");

#FIP is a way of designating counties (https://en.wikipedia.org/wiki/List_of_FIPS_codes_for_Georgia_(U.S._state))
PatchFIPS = GA_county_size.FIP
PatchCounty = GA_county_size.County
#NOTE - will assign initial states by GA_county_size
#USE THE FIP indices here to keep everything ordered.

#Additional data from https://dph.georgia.gov/covid-19-daily-status-report
#Note not sure any of this information is 
#Obtained 26 March 2020
#PropCasesHosp = 0.2762 #0.2829 noon 27/3#0.3098 7pm 26/3# 0.3102 noon26/3/20# 31%
#PropCasesDead = 0.0296 #0.032 noon 27/3#0.0341 7pm26/3# 0.0315 noon26/3/20# 3.1%
#0-17; 18-59; 60+;Unknown
#PropAge = [0.01,0.59,0.35,0.05] #confirm these add to 1 - Yes! noon 26/3/20
#PropAge = [0.01, 0.57, 0.35, 0.07] #7pm 26/3/20
#PropAge = [0.01,0.57,0.34,0.08]#noon 27/3
#PropAge = [0.01,0.56,0.34,0.09]#7pm 27/3

#map unknowns evenly across age groups
#MapToAgeCategories
#0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80+
#PropCaseAge = [0.5*PropAge[1],0.5*PropAge[1],0.25*PropAge[2],0.25*PropAge[2],0.25*PropAge[2],0.25*PropAge[2],(1/3)*PropAge[3],(1/3)*PropAge[3],(1/3)*PropAge[3]]
#PropCaseAge = PropCaseAge./sum(PropCaseAge); #map unknowns evenly across age groups


#Make plot of the age-distribution across counties
makeplot=0
if makeplot==1
gr()
plot(MiddleAge,100 .* convert(Array,(GA_county_size[1,3:11])),xlim=(0,100),title="GA county demogaphy",
    ylabel="Pop. Demography (%)",xlabel="Age",legend=false,yguidefontsize=8,markershape = :auto,markersize=2)
for aa=2:159
    plot!(MiddleAge,100 .* convert(Array,(GA_county_size[aa,3:11])),markershape = :auto,markersize=2)
end
savefig("GAagestructure.pdf")    
current()
end


In [36]:
### quick note on 10th April ###
#Here we load in origin-destination data (159x159) showing trip numbers between places (including self-loops) which is sampled (not normalised)
#we need to translate into a form useful to the model
#to do this we want to:
#a) normalise such that we know the proportion of trips from an origin to a destination
#b) replace the diagonal (self trips) with negative elements that sum to the proportion of those leaving
#c) transpose the data such that we can use it in a matrix multiplication to calculate net pop. change (- leavers + incomers) for each type of state e.g. multiply transport matrix by all S individuals in age class 20-29 (159x1) [ then repeat for all epi states and age classes]
#we are going to get transport matrices w for each of the age structured commuting data
### quick note ends ###

#load and create commuter matrix
#############Many Ways To Do This ##################
#Have to start somewhere
#Later:
#1) Add in extra commuting pattern data
#2) Think about how physical distancing,..., public policy may affect commuting patterns
####################################################
COMMRULES = OD30_55[:,2:end] #matrix of data showing origin (row) and destination (col) patterns by county.
#data is total number of trips from a field sampling approach. Want to turn into a proportion.
#Note 1: this is a sample. Needs normalising. Note 2: first column was FIP
JJ = 1.0*convert(Array,COMMRULES)#convert to matrix of Floats
sJJ = sum(JJ,dims=1) #sum over columns - this is the total arrivals (including self)
for aa =1:159
    JJ[:,aa] = JJ[:,aa] ./ sJJ[aa]
end
#find proportion of those staying in county
PROPSTAY = JJ[1:size(JJ, 1)+1:end]
#reassign diagonal elements as those who will be leaving county
JJ[1:size(JJ, 1)+1:end] = -(1.0 .- PROPSTAY)
#transpose matrix as we require destination to be the rows and origin to be the columns
w = JJ';

COMMRULES = ODL29[:,2:end] #matrix of data showing origin (row) and destination (col) patterns by county.
#Note 1: this is a sample. Needs normalising. Note 2: first column was FIP
JJ = 1.0*convert(Array,COMMRULES)#convert to matrix of Floats
sJJ = sum(JJ,dims=1) #sum over columns - this is the total arrivals (including self)
for aa =1:159
    JJ[:,aa] = JJ[:,aa] ./ sJJ[aa]
end
#find proportion of those staying in county
PROPSTAY = JJ[1:size(JJ, 1)+1:end]
#reassign diagonal elements as those who will be leaving county
JJ[1:size(JJ, 1)+1:end] = -(1.0 .- PROPSTAY)
#transpose matrix as we require destination to be the rows and origin to be the columns
wy = JJ';


COMMRULES = OD55_up[:,2:end] #matrix of data showing origin (row) and destination (col) patterns by county.
#Note 1: this is a sample. Needs normalising. Note 2: first column was FIP
JJ = 1.0*convert(Array,COMMRULES)#convert to matrix of Floats
sJJ = sum(JJ,dims=1) #sum over columns - this is the total arrivals (including self)
for aa =1:159
    JJ[:,aa] = JJ[:,aa] ./ sJJ[aa]
end
#find proportion of those staying in county
PROPSTAY = JJ[1:size(JJ, 1)+1:end]
#reassign diagonal elements as those who will be leaving county
JJ[1:size(JJ, 1)+1:end] = -(1.0 .- PROPSTAY)
#transpose matrix as we require destination to be the rows and origin to be the columns
wo = JJ';




In [37]:
#Basic epidemiology to apply across all patches
SocialDistancingInterventions=[1.0,0.75,0.6,0.4] #proportionalities to apply to both β_A and β_S (1.0 is no distancing)
β_A = 0.3 #asymptomatic transmission rate (per day)
β_S = 0.6 #symptomatic transmission rate (per day)
γ_e = (1.0/4.0) #mean exposed period (days)
γ_a = (1.0/6.0) #mean asymptomatic period (days)
γ_s = (1.0/6.0) #mean symptomatic period (days)
γ_h = (1.0/10.0) # mean hospitalised period (days)

p = [0.95, 0.95,0.9,0.8,0.7,0.6,0.4,0.2,0.2]  #fraction of cases that are asymptomatic
h = [0.1, 0.3, 1.2, 3.2, 4.9, 10.2, 16.6, 24.3, 27.3]./100; #fraction of cases that require a hospital visit (by age class)
ξ = [5, 5, 5, 5, 6.3, 12.2, 27.4, 43.2, 70.9]./100; #fraction of hospitalised cases that become critical and require critical care (by age class)
μ = 0.5.*ones(NumAgeClasses) #mortality fraction

ISMOVEFACTOR = 1.0 # how much to let Is people commute (must be 0-1)
w1 = 16/24 ; #fraction of time mixing with local pop.
w2 = 1.0 - w1 ;#fraction of time mixing with commuter pop.


In [38]:
#SPACE = GA_pop_ar.Area_SqMI;
#scatter(PROPSTAY,GA_county_size.POPSIZE ./ SPACE,markershape = :auto,markersize=2, yaxis=:log)
#scatter!(PROPSTAY,GA_county_size.POPSIZE)

In [39]:
#set parameter sets to explore

param = [1.0,β_A,β_S,p,γ_e,γ_a,γ_s,γ_h,h,ξ,μ,S_ind,E_ind,
    Ia_ind,Is_ind,Ihsub_ind,Ihcrit_ind,R_ind,D_ind,
    NumPatches,NumAgeClasses,patchstates,patchstatesALIVE,
    patchstatesMOVEABLE,w,wy,wo,transport,ISMOVEFACTOR,w1,w2]

singlenodepar =[1.0,β_A,β_S,p,γ_e,γ_a,γ_s,γ_h,h,ξ,μ,S_ind,E_ind,Ia_ind,Is_ind,R_ind,Ihsub_ind,Ihcrit_ind,D_ind]


19-element Array{Any,1}:
 1.0                                                                                                  
 0.3                                                                                                  
 0.6                                                                                                  
  [0.95, 0.95, 0.9, 0.8, 0.7, 0.6, 0.4, 0.2, 0.2]                                                     
 0.25                                                                                                 
 0.16666666666666666                                                                                  
 0.16666666666666666                                                                                  
 0.1                                                                                                  
  [0.001, 0.003, 0.012, 0.032, 0.049, 0.102, 0.166, 0.243, 0.273]                                     
  [0.05, 0.05, 0.05, 0.05, 0.063, 0.122, 0.27399

 ### Set the initial conditions
 
We now have the ingredients. Now we need to mix in the ingredients.
Want to set initial conditions, being cognisant of the GA data.
Two main ways right now are: a simulation approach (CHOICE=5); and an "eigenvector" approach (CHOICE=7)

Simulation starts an epidemic in Fulton & Dougherty, then runs the model forward until deaths match up those recorded in GA data for each county one-by-one. The states in each county at these times are then patched together into an initial condition.

Eigenvector approach runs the model at the Georgia level till total deaths are matched. Then assigns the final vector of values proportionally across counties as an intial condition.

In [40]:

#Assign county age structured data to initial state 
GAPOP = sum(GA_county_size.POPSIZE) #total population of GA

MAT = convert(Array, (GA_county_size[:,3:(end-1)] .* GA_county_size.POPSIZE))
MAT2 = MAT ./ GAPOP
#sum(MAT2)#should = 1. Yes!
#MAT2 is the fraction of GA in eaah age class in each county.

#want to now assign MAT2 to initial vector u00
u00 = zeros(NumPatches*NumAgeClasses*NumEpiStates)

for aa = 1:NumPatches
        u00[(aa-1)*patchstates .+ S_ind] = MAT2[aa,:]
end


# load case data and figure out initial conditions

tspan = (0.0,300.0)
#start epidemic with 1 exposed individual ages 20-29 in Fulton county
INDFult = findall(GA_county_size.FIP .== 13121)
INDFult=INDFult[1]#patch index
u00[S_ind[3] + patchstates*(INDFult-1)] = u00[S_ind[3] + patchstates*(INDFult-1)] - 1.0/GAPOP #minus from S
u00[E_ind[3] + patchstates*(INDFult-1)] = u00[E_ind[3] + patchstates*(INDFult-1)] + 1.0/GAPOP #add to E
INDDougherty = findall(GA_county_size.FIP .== 13095) #dougherty county (Albany)
INDDougherty=INDDougherty[1]#patch index
u00[S_ind[3] + patchstates*(INDDougherty-1)] = u00[S_ind[3] + patchstates*(INDDougherty-1)] - 1.0/GAPOP #minus from S
u00[E_ind[3] + patchstates*(INDDougherty-1)] = u00[E_ind[3] + patchstates*(INDDougherty-1)] + 1.0/GAPOP #add to E
initu = u00
prob_init = ODEProblem(SpatialAgeStructuredEpidemiologicalModelMixAge!,initu,tspan,param)


affect!(integrator) = terminate!(integrator)

    #state wide deaths and cases
TOT_deaths = sum(CasesbyCounty.deaths)/GAPOP
TOT_cases = sum(CasesbyCounty.cases)/GAPOP



if CHOICES == 1



    #set up callback to stop integration when number of cases reaches a threshold
    CaseThreshold = TOT_deaths
    
    #sum all D everywhere
    condition = function(u,t,integrator)
            D_ind = (7*9 +1 ):(8*9) #d ind in first patch
            patchstates = 8*9 #num of states per patch 
            sumD = 0.0
            for aa = 1:159 #for each patch
                sumD += sum(u[D_ind .+ patchstates*(aa-1)])
            end
            return CaseThreshold - sumD
        end
    
    
    cb = ContinuousCallback(condition, affect!)
    #solve the initial spin up, checking for positivity and using callback definition 
    sol_init = solve(prob_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=0.1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb)
    END = sol_init.u[end]
    u00 = END

elseif CHOICES==2
    #cases county by county
    for aa = 1:length(GA_county_size.FIP) #for each county in GA
    print(aa)
        IND = findall(CasesbyCounty.FIP .== GA_county_size.FIP[aa]) #find patch index
        if length(IND) == 1 #ignore unknown category in case data
            CASES = 1.0 * CasesbyCounty.cases[IND[1]]
        else
            CASES = 1.0
        end
            #set up callback to stop integration when number of cases reaches a threshold
            CaseThreshold = CASES/GAPOP
            #sum of S in patch - (all in patch - thresh)
            condition(u,t,integrator)  = sum(u[[S_ind;E_ind;Ia_ind] .+ patchstates*(aa-1)]) - ( sum(u[(1:patchstates) .+ patchstates*(aa-1)]) - CaseThreshold )  #this must be equal to zero at stopping condition
            affect!(integrator) = terminate!(integrator) 
            cb = ContinuousCallback(condition,affect!)
            #solve the initial spin up, checking for positivity and using callback definition 
            sol_init = solve(prob_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb)
            #assign end spinup states per county into new initial condition.
            END = sol_init.u[end]
            u00[(1+patchstates*(aa-1)):(patchstates*aa)] = END[(1+patchstates*(aa-1)):(patchstates*aa)]
    end

elseif CHOICES == 3
    #deaths by county (if no deaths - cases)
    

    for aa = 1:length(GA_county_size.FIP) #for each county in GA
    print(aa)
        IND = findall(CasesbyCounty.FIP .== GA_county_size.FIP[aa]) #find patch index
        if length(IND) == 1 #ignore unknown category in case data
            CASES = 1.0 * CasesbyCounty.cases[IND[1]] / GAPOP
            DEATHS = 1.0 * CasesbyCounty.deaths[IND[1]] / GAPOP
        else
            CASES = 10.0 / GAPOP
            DEATHS = 0.0
        end

        
        if DEATHS == 0.0 #assign based on cases
            #set up callback to stop integration when number of cases reaches a threshold
        CaseThreshold = CASES
            #sum of S in patch - (all in patch - thresh)
            condition(u,t,integrator)  = (sum(u[(1:patchstates) .+ patchstates*(aa-1)]) - sum(u[[S_ind;E_ind;Ia_ind;Is_ind] .+ patchstates*(aa-1)]))  - CaseThreshold   #this must be equal to zero at stopping condition
            affect!(integrator) = terminate!(integrator) 
            cb = ContinuousCallback(condition,affect!)
            #solve the initial spin up, checking for positivity and using callback definition 
            sol_init = solve(prob_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb)
            #assign end spinup states per county into new initial condition.
            END = sol_init.u[end]
            u00[(1+patchstates*(aa-1)):(patchstates*aa)] = END[(1+patchstates*(aa-1)):(patchstates*aa)]     
        else #assign based on deaths
            #set up callback to stop integration when number of cases reaches a threshold
        CaseThreshold = DEATHS
            #sum of S in patch - (all in patch - thresh)
            condition2(u,t,integrator)  = sum(u[D_ind .+ patchstates*(aa-1)]) - CaseThreshold   #this must be equal to zero at stopping condition
            affect2!(integrator) = terminate!(integrator) 
            cb2 = ContinuousCallback(condition2,affect2!)
            #solve the initial spin up, checking for positivity and using callback definition 
            sol_init = solve(prob_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb2)
            #assign end spinup states per county into new initial condition.
            END = sol_init.u[end]
            u00[(1+patchstates*(aa-1)):(patchstates*aa)] = END[(1+patchstates*(aa-1)):(patchstates*aa)]
        end
            
    end

elseif CHOICES == 4
    #Simple model to total deaths at county level. If no data on deaths - fill as the highest of when Fulton is hit (maybe should also consider an Albany start).
    

    for aa = 1:length(GA_county_size.FIP) #for each county in GA
    print(aa)
        flag=0
        IND = findall(CasesbyCounty.FIP .== GA_county_size.FIP[aa]) #find patch index
        if length(IND) == 1 #ignore unknown category in case data
            CASES = 1.0 * CasesbyCounty.cases[IND[1]] / GAPOP
            DEATHS = 1.0 * CasesbyCounty.deaths[IND[1]] / GAPOP
        else
            CASES = 1.0 / GAPOP
            IND2 = findall(CasesbyCounty.FIP .== 13121)
            DEATHS = 1.0 * CasesbyCounty.deaths[IND2[1]] / GAPOP
            flag=1
        end

        
        if flag==1 #assign based on cases
            #set based on Fulton deaths
        CaseThreshold = DEATHS
            #sum of S in patch - (all in patch - thresh)
            condition(u,t,integrator)  =  sum(u[D_ind .+ patchstates*(INDFult-1)]) - CaseThreshold   #this must be equal to zero at stopping condition
            affect!(integrator) = terminate!(integrator) 
            cb = ContinuousCallback(condition,affect!)
            #solve the initial spin up, checking for positivity and using callback definition 
            sol_init = solve(prob_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb)
            #assign end spinup states per county into new initial condition.
            END = sol_init.u[end]
            u00[(1+patchstates*(aa-1)):(patchstates*aa)] = END[(1+patchstates*(aa-1)):(patchstates*aa)]     
        else #assign based on deaths
            #set up callback to stop integration when number of cases reaches a threshold
        CaseThreshold = DEATHS
            #sum of S in patch - (all in patch - thresh)
            condition2(u,t,integrator)  = sum(u[D_ind .+ patchstates*(aa-1)]) - CaseThreshold   #this must be equal to zero at stopping condition
            affect2!(integrator) = terminate!(integrator) 
            cb2 = ContinuousCallback(condition2,affect2!)
            #solve the initial spin up, checking for positivity and using callback definition 
            sol_init = solve(prob_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb2)
            #assign end spinup states per county into new initial condition.
            END = sol_init.u[end]
            u00[(1+patchstates*(aa-1)):(patchstates*aa)] = END[(1+patchstates*(aa-1)):(patchstates*aa)]
        end
            
    end    
    

elseif CHOICES ==5
    #hospitalisations rather than cases, unless death available
    Hprop = 0.248 #20.48% hosp
      #cases county by county
    for aa = 1:length(GA_county_size.FIP) #for each county in GA
    print(aa)
        IND = findall(CasesbyCounty.FIP .== GA_county_size.FIP[aa]) #find patch index
        DEATH = 0.0
        if length(IND) == 1 #ignore unknown category in case data
            DEATH = CasesbyCounty.deaths[IND[1]]
            HOSP = Hprop * CasesbyCounty.cases[IND[1]]
        else
            HOSP = 0.5
        end
        if DEATH >0.0
             #set up callback to stop integration when number of cases reaches a threshold
            CaseThreshold = DEATH/GAPOP
            #sum of S in patch - (all in patch - thresh)
            condition(u,t,integrator)  = sum(u[D_ind .+ patchstates*(aa-1)]) - CaseThreshold  #this must be equal to zero at stopping condition
            affect!(integrator) = terminate!(integrator) 
            cb = ContinuousCallback(condition,affect!)
            #solve the initial spin up, checking for positivity and using callback definition 
            sol_init = solve(prob_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb)
            #assign end spinup states per county into new initial condition.
            END = sol_init.u[end]
            u00[(1+patchstates*(aa-1)):(patchstates*aa)] = END[(1+patchstates*(aa-1)):(patchstates*aa)]
        else
            #set up callback to stop integration when number of cases reaches a threshold
            CaseThreshold = HOSP/GAPOP
            #sum of S in patch - (all in patch - thresh)
            condition2(u,t,integrator)  = sum(u[[Ihsub_ind;Ihcrit_ind] .+ patchstates*(aa-1)]) - CaseThreshold  #this must be equal to zero at stopping condition
            affect2!(integrator) = terminate!(integrator) 
            cb2 = ContinuousCallback(condition2,affect2!)
            #solve the initial spin up, checking for positivity and using callback definition 
            sol_init = solve(prob_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb2)
            #assign end spinup states per county into new initial condition.
            END = sol_init.u[end]
            u00[(1+patchstates*(aa-1)):(patchstates*aa)] = END[(1+patchstates*(aa-1)):(patchstates*aa)]
        end
    end
   
    
elseif CHOICES == 6
    #Simple model to total deaths at state level. Then reapportion in the county model.
    Singleu0 = zeros(NumAgeClasses*NumEpiStates)
    for aa =1:159
        for bb = 1:9
        Singleu0[S_ind[bb]] += initu[S_ind[bb] .+ patchstates*(aa-1)]
        end
    end
    Singleu0[E_ind[3]] = 1.0/GAPOP  #seed the epidemic (1 person 20-29).
    print(sum(Singleu0[1:9]))
    
    prob_s_init = ODEProblem(SingleNode!,Singleu0,tspan,singlenodepar)
    CaseThreshold = TOT_deaths
    #print(CaseThreshold)
    condition3(u,t,integrator)  = CaseThreshold - sum(u[D_ind])   #this must be equal to zero at stopping condition
    affect3!(integrator) = terminate!(integrator) 
    cb3 = ContinuousCallback(condition3,affect3!)
    #solve the initial spin up, checking for positivity and using callback definition 
    sol_s_init = solve(prob_s_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=0.1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb3)
    START = sol_s_init.u[end] #initial distributions at state scale 80(age*epi)
    #mat2 = patch*age
    #partition these distributions across counties
    
    D_prop = zeros(159)
    HAM=0
    for aa = 1:159
         IND = findall(CasesbyCounty.FIP .== GA_county_size.FIP[aa]) #find patch index
        if length(IND) == 1 #ignore unknown category in case data
            D_prop[aa] = 1.0 * CasesbyCounty.deaths[IND[1]]
        end
    end
    INDY = findall(D_prop == 0.0)
    for bb = 1:length(INDY)
    D_prop[INDY[bb]] = 1.0/(length(INDY))
    end
    print(sum(D_prop))
    D_prop = D_prop ./GAPOP
    
    D_prop = D_prop / 9 # (age classes)
        
        
    
    for aa=1:159
        for bb = 1:9
        initu[S_ind[bb] + patchstates*(aa-1)] = START[S_ind[bb]] .* MAT2[aa,bb] * D_prop[aa]
        initu[E_ind[bb] + patchstates*(aa-1)] = START[E_ind[bb]] .* MAT2[aa,bb] * D_prop[aa]
        initu[Ia_ind[bb] + patchstates*(aa-1)] = START[Ia_ind[bb]] .* MAT2[aa,bb]* D_prop[aa]
        initu[Is_ind[bb] + patchstates*(aa-1)] = START[Is_ind[bb]] .* MAT2[aa,bb]* D_prop[aa]
        initu[R_ind[bb] + patchstates*(aa-1)] = START[R_ind[bb]] .* MAT2[aa,bb]* D_prop[aa]
        initu[Ihsub_ind[bb] + patchstates*(aa-1)] = START[Ihsub_ind[bb]] .* MAT2[aa,bb]* D_prop[aa]
        initu[Ihcrit_ind[bb] + patchstates*(aa-1)] = START[Ihcrit_ind[bb]] .* MAT2[aa,bb]* D_prop[aa]
        initu[D_ind[bb] + patchstates*(aa-1)] = START[D_ind[bb]] .* MAT2[aa,bb]* D_prop[aa]
        end
    end
    
    
    u00 = initu
    
elseif CHOICES ==7
    
    #1.
    #Simple model to total deaths at state level. Then reapportion in the county model.
    Singleu0 = zeros(NumAgeClasses*NumEpiStates)
    Singleu0[S_ind] = sum(MAT2,dims=1)
    Singleu0[S_ind[3]] = Singleu0[S_ind[3]] - 1.0/GAPOP
    Singleu0[E_ind[3]] = 1.0/GAPOP  #seed the epidemic (1 person 20-29).


    prob_s_init = ODEProblem(SingleNode!,Singleu0,tspan,singlenodepar)

    CaseThreshold = TOT_deaths

    condition3(u,t,integrator)  = CaseThreshold - sum(u[D_ind])   #this must be equal to zero at stopping condition
    affect3!(integrator) = terminate!(integrator) 
    cb3 = ContinuousCallback(condition3,affect3!)

    #solve the initial spin up, checking for positivity and using callback definition 
    sol_s_init = solve(prob_s_init,Tsit5(), abstol=1e-8,reltol=1e-8,saveat=0.1,isoutofdomain=(y,p,t)->any(x->x<0,y), callback = cb3)

    #2.
    START = sol_s_init.u[end] #initial distributions at state scale 80(age*epi)
    SumD = sum(START[D_ind])*GAPOP
    print(SumD)
    TOTD =  sum(CasesbyCounty.deaths)
    TOTU =  CasesbyCounty.deaths[CasesbyCounty.FIP.==-999][1]
    L1 = length(GA_county_size.FIP)
    L2 = length(CasesbyCounty.FIP)-1
    
    for aa = 1:159
        countypop = GA_county_size.POPSIZE[aa]
        
        IND = findall(CasesbyCounty.FIP .== GA_county_size.FIP[aa]) #find patch index
        if length(IND) == 1 #ignore unknown category in case data
            D_county = 1.0 * CasesbyCounty.deaths[IND[1]]
        else
            D_county = TOTU/(L1-L2)
        end
        
        RATIO = 1.0*(D_county/TOTD) #county ratio

        NEW = START .* RATIO
        JAM = zeros(NumAgeClasses)
        
        for bb = 1:NumAgeClasses
            JAM[bb]=sum(NEW[[E_ind[bb];Ia_ind[bb];Is_ind[bb];R_ind[bb];Ihsub_ind[bb];Ihcrit_ind[bb];D_ind[bb]]])
        end
        
        
        AgeFrac = MAT[aa,:] #numppl in county in each age
      
        NEW[S_ind] = (AgeFrac .- (JAM .* countypop))/GAPOP
        
        if sum(isless.(NEW,0)) < 0
            print("WARNING. u LESS THAN 0")
            NEW[isless.(NEW,0)] = 0.0
        end

        u00[(1+patchstates*(aa-1)):(patchstates*aa)] = NEW

    end
    
end

#assign end spinup states as new initial condition.


#sde = 0.0
#for aa = 1:159
#sde = sde + sum(u00[D_ind .+ patchstates*(aa-1)])
#end
#sde*GAPOP
num=0
for aa = 1:159
num += sum(u00[D_ind .+ patchstates*(aa-1)])
end
num*GAPOP
print("/n",sum(u00))
print(num*GAPOP)

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159/n1.0391.02933056302606

In [41]:
num=0
for aa = 1:159
num += sum(u00[D_ind .+ patchstates*(aa-1)])
    print(string(sum(u00[D_ind .+ patchstates*(aa-1)])*GAPOP),"\n")
end
num*GAPOP

aa=4
IND = findall(CasesbyCounty.FIP .== GA_county_size.FIP[aa])

0.32533165005430814
0.20898595783198567
0.6575232528065296
1.999999999999997
0.9999999999999996
0.20158166918130263
2.999999999999995
11.999999999999998
0.24152089225438028
0.15137520480586109
0.9999999999999997
0.025653707403262647
0.9999999999999993
0.9999999999999994
1.999999999999997
0.48116307213653303
0.8195619710776098
1.0081880452298406
1.9999999999999951
0.6728488035158914
0.11313302801628618
3.9999999999999996
0.2590397413574327
0.03232637563514231
3.9999999999999996
0.0655331724945101
0.9999999999999979
5.999999999999998
8.99999999999999
0.999999999999999
10.999999999999968
0.07714638812880584
28.999999999999982
1.9999999999999982
4.999999999999999
1.2486469869555827
0.07664383602835721
1.9999999999999987
0.03913508815588764
1.7046794266201368
0.9999999999999997
0.9999999999999993
0.485263210963761
11.999999999999996
0.1588281269858178
0.9999999999999986
61.999999999999964
4.999999999999996
4.999999999999999
0.0439167039807445
0.9999999999999999
0.03197455037642326
0.0860672

1-element Array{Int64,1}:
 86

In [42]:
#clean up memory by removing tables and objects no longer of interest
ODL29 = 0
OD30_55 = 0
OD55_up = 0
#GA_county_size = 0
#CasesbyCounty = 0
MAT = 0
MAT2 = 0
JJ =0


0

### Run the simulations

Now that the initial conditions are set, we are ready to bake the model.
Note that in the above conditions are set without any social distancing interventions. Currently, these are applied at time =0. That is here.
Here, we have a loop that goes through different social distancing levels (that reduce βS and βA); runs the model and saves the output to file.

In [43]:
#NOW DO INTERVENTIONS IN A LOOP

mkdir(SAVEFOLD)
for bb = 1:length(SocialDistancingInterventions)
    
    SAVEFOLD2 = string(SAVEFOLD,"/Social_",bb)
    
    #set parameters for this intervention
    pars = [SocialDistancingInterventions[bb],β_A,β_S,p,γ_e,γ_a,γ_s,γ_h,h,ξ,μ,S_ind,E_ind,
    Ia_ind,Is_ind,Ihsub_ind,Ihcrit_ind,R_ind,D_ind,
    NumPatches,NumAgeClasses,patchstates,patchstatesALIVE,
    patchstatesMOVEABLE,w,wy,wo,transport,ISMOVEFACTOR,w1,w2]
   
    
    #define ODE problems to solve
    prob = ODEProblem(SpatialAgeStructuredEpidemiologicalModelMixAge!,u00,tspanrun,pars)
    #set New Problem    
    #solve the ODEs
    SOL = solve(prob,Tsit5(),abstol=1e-8,reltol=1e-8,saveat=TIMERESOLUTION,isoutofdomain=(y,p,t)->any(x->x<0,y))
 
    #save output

    WriteOutput!(PatchFIPS,PatchCounty,GAPOP,TODAY,TIMERESOLUTION,T_END,SOL,pars,SAVEFOLD2)
    
    
    JAM = GAPOP * gatherStates!(SOL,param)
    
    
    #county-level projected deaths
    gr()
    plot(SOL.t,dropdims(sum(JAM[1,8,1:9,:],dims=1),dims=1),legend=false,ylabel="D per county",xlabel="Time (days)")

    for aa = 2:159
        plot!(SOL.t,dropdims(sum(JAM[aa,8,1:9,:],dims=1),dims=1))
    end
    savefig(string(SAVEFOLD2,"/deaths.png"))
    #current()

    #county-level projected ICU
    plot(SOL.t,dropdims(sum(JAM[1,7,1:9,:],dims=1),dims=1),legend=false,ylabel="ICU per county",xlabel="Time (days)")

    for aa = 2:159
        plot!(SOL.t,dropdims(sum(JAM[aa,7,1:9,:],dims=1),dims=1))
    end
    savefig(string(SAVEFOLD2,"/ICU.png"))
    #current()

    #county-level projected hosp.s
    plot(SOL.t,dropdims(sum(JAM[1,6,1:9,:],dims=1),dims=1).+dropdims(sum(JAM[1,7,1:9,:],dims=1),dims=1),legend=false,ylabel="Hosp. per county",xlabel="Time (days)")

    for aa = 2:159
        plot!(SOL.t,dropdims(sum(JAM[aa,6,1:9,:],dims=1),dims=1).+dropdims(sum(JAM[1,7,1:9,:],dims=1),dims=1))
    end
    savefig(string(SAVEFOLD2,"/totalHosp.png"))
    #current()

    p1 = SummaryPlot!(JAM,SOL.t)
    savefig(string(SAVEFOLD2,"/GAscenarios.pdf"))
    #current()
  
    
    
end

    